In [1]:
#import the package
import pandas as pd
import numpy as np
import time
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [2]:
transaction=pd.read_csv('2021 Year End Transaction List.csv')
prod_map = pd.read_csv('ProductMapping.csv')
new_customer = pd.read_csv('2021 Year End Customer List.csv')
overhead_coupon = pd.read_excel('2021 Amazon Transactions - Seller Central.xlsx', \
                                sheet_name= '2021Jan1-2021Dec31CustomUnified')

In [3]:
# Convert the date to datetime64
transaction['Date'] = pd.to_datetime(transaction['Date'], format='%d/%m/%Y')

In [4]:
# Filter data between two dates
#trans_this_year= transaction.loc[transaction['Date'] >= '2021-01-01']
trans_this_year= transaction.copy()

In [5]:
trans_this_year['AmazonFee'] = trans_this_year['ProductSales'] - trans_this_year['Total']

In [6]:
trans_this_year['month'] = trans_this_year['Date'].dt.month

In [ ]:
#Sales by SKU
sales_sku = pd.pivot_table(trans_this_year, index ='Sku', columns = 'month', values = 'ProductSales', \
                              aggfunc = {'ProductSales': np.sum})
sales.to_csv('sales_sku.csv')

In [22]:
trans_product = trans_this_year.merge(prod_map, how= 'left', on= 'Sku')

In [ ]:
#Sales by Product
sales_product = pd.pivot_table(trans_this_year, index ='ProductLine', columns = 'month', values = 'ProductSales', \
                              aggfunc = {'ProductSales': np.sum})
sales_product.to_csv('sales_product.csv')

In [8]:
new_customer['FirstPurchaseDate'] = pd.to_datetime(new_customer['FirstPurchaseDate'], format='%d/%m/%Y')

In [23]:
trans_product['concat']  = trans_product['ClientId'].astype(str) + trans_product['ProductLine']
july= trans_product[['Date', 'ClientId', 'ProductLine', 'concat']]
new_july = july.groupby(['ClientId']).agg(Minimum_Date=('Date', np.min)).reset_index()

In [24]:
new_july

,ClientId,Minimum_Date
0,1.0,2017-12-16
1,2.0,2021-05-28
2,3.0,2017-11-06
3,4.0,2017-11-09
4,6.0,2017-10-25
...,...,...
142617,145167.0,2021-12-31
142618,145168.0,2021-12-31
142619,145169.0,2021-12-31
142620,145170.0,2021-12-31


In [25]:
new_july = new_july.merge(july, how='inner', left_on=['ClientId', 'Minimum_Date'], right_on=['ClientId', 'Date'])

In [35]:
sample = new_july.loc[new_july['ClientId'] == 93985]

In [36]:
sample

,ClientId,Minimum_Date,Date,ProductLine,concat
94130,93985.0,2021-07-01,2021-07-01,Yak Chew,93985.0Yak Chew


In [28]:
#Average 180 days sales of new customer
new_july =new_july[(new_july['Minimum_Date'] >= '2021/07/01') &\
                       (new_july['Minimum_Date'] <= '2021/07/31')]

new_client_list = new_july['concat'].to_list()
new_customer_sales = trans_product[trans_product['concat'].isin(new_client_list)]
#new_customer_sales = new_customer_sales[(new_customer_sales['Date'] >= '2021/07/01') & \
                                       #(new_customer_sales['Date'] <= '2021/07/31')]
average_180_days = pd.pivot_table(new_customer_sales, index =['ClientId', 'ProductLine'], values = 'ProductSales', \
                              aggfunc = {'ProductSales': np.sum})

average_180_days.to_csv('average_180_days.csv')

In [13]:
#new_july.to_csv('new_july.csv')
new_customer_sales.to_csv('new_customer_sales.csv')

In [37]:
new_cust_added = new_customer.merge(trans_this_year, how= 'left', on= 'ClientId')

In [38]:
new_cust_added = new_cust_added.drop_duplicates(subset= 'ClientId', keep= 'first', inplace= True)

In [ ]:
#New Customer by Product Line
new_cust_product = pd.pivot_table(new_cust_added, index ='ProductLine', column= 'month', values = 'ClientId', \
                              aggfunc = 'new_cust_added')

new_cust_product('new_cust_product.csv', index= False)

In [ ]:
#New Customer by SKU
new_cust_sku = pd.pivot_table(new_cust_added, index ='Sku', column= 'month', values = 'ClientId', \
                              aggfunc = 'count')

new_cust_sku('new_cust_sku.csv', index= False)

In [ ]:
#Fees by Product Line
product_fee = pd.pivot_table(trans_this_year, index ='ProductLine', columns = 'month', values = 'AmazonFee', \
                              aggfunc = {'AmazonFee': np.sum})

product_fee.to_csv('product_fee.csv', index= False)

In [ ]:
#Fees by SKU
sku_fee = pd.pivot_table(trans_this_year, index ='Sku', columns = 'month', values = 'AmazonFee', \
                              aggfunc = {'AmazonFee': np.sum})

sku_fee.to_csv('sku_fee.csv', index= False)